# Motivation
Imagine you are a manager of a warehouse. You need to prepare racks and pallets for storing items. So how many racks and pallet you need to plan for your organization?

# Introduction to PuLP
Linear programming (LP) is one of the best methods to find optimal solutions for problems with constraints like the above. PuLP is a Python library that makes it easy to apply linear programming using Python.

In [1]:
import pandas as pd
import pulp
from itertools import product
from termcolor import colored

In [2]:
d = {
    'definition': ['Stock A', 'Stock B', 'Rack capacity for A', 'Rack capacity for B',
                  'Pallet capacity for A', 'Pallet capacity for B', 'Minimum total nb of racks', 'Minimum total nb of pallets'],
    'unit': ['item', 'item', 'item', 'item', 'item', 'item', 'rack', 'pallet'],
    'var': ['Da', 'Db', 'Cr_a', 'Cr_b', 'Cp_a', 'Cp_b', 'NB r', 'NB p'],
    'value': [1074887, 22080, 7911, 561, 375, 58, 954, 785]
}
df = pd.DataFrame(data=d)
df

,definition,unit,var,value
0,Stock A,item,Da,1074887
1,Stock B,item,Db,22080
2,Rack capacity for A,item,Cr_a,7911
3,Rack capacity for B,item,Cr_b,561
4,Pallet capacity for A,item,Cp_a,375
5,Pallet capacity for B,item,Cp_b,58
6,Minimum total nb of racks,rack,NB r,954
7,Minimum total nb of pallets,pallet,NB p,785


## Formulation
Input parameters:
- Number of stock (D)
- Capacity of rack and pallet (C)
- Number of rack and pallet (NB)

With rack, pallet with item A, B we will have combination of variables:

In [3]:
var = df.set_index('var').to_dict()['value']
var

{'Da': 1074887,
 'Db': 22080,
 'Cr_a': 7911,
 'Cr_b': 561,
 'Cp_a': 375,
 'Cp_b': 58,
 'NB r': 954,
 'NB p': 785}

## Transform into pulp library

In [4]:
# dynamic variables
pack_type = ['r', 'p']
product_bin = ['a', 'b']
lst = [f'{p}_{b}' for p, b in product(pack_type, product_bin)]
print(f'number of racks, pallets for item a, b: {lst}')

number of racks, pallets for item a, b: ['r_a', 'r_b', 'p_a', 'p_b']


In [5]:
# minimize problem
x = pulp.LpVariable.dicts("x", lst, cat='Continuous', lowBound=0)
model = pulp.LpProblem("warehouse_optimization", pulp.LpMinimize)

# objective
model += sum([x[i]for i in lst])

# capacities
for b in product_bin:
    model += sum(var[f'C{r}_{b}'] * x[f'{r}_{b}'] for r in pack_type) >= var[f'D{b}']

# total number of racks, pallets
for r in pack_type:
    model += sum(x[f'{r}_{b}'] for b in product_bin) >= var[f'NB {r}']

You can double check what you have written to your model as aligned with your fomulation

In [6]:
model

warehouse_optimization:
MINIMIZE
1*x_p_a + 1*x_p_b + 1*x_r_a + 1*x_r_b + 0
SUBJECT TO
_C1: 375 x_p_a + 7911 x_r_a >= 1074887

_C2: 58 x_p_b + 561 x_r_b >= 22080

_C3: x_r_a + x_r_b >= 954

_C4: x_p_a + x_p_b >= 785

VARIABLES
x_p_a Continuous
x_p_b Continuous
x_r_a Continuous
x_r_b Continuous

In [7]:
# solve
model.solve(pulp.PULP_CBC_CMD(msg=False))
text = f'status: {pulp.LpStatus[model.status]}'
color = 'blue' if model.status == 1 else 'red'
print(colored(text, color, attrs=['reverse', 'blink']))

status: Optimal


In [8]:
result = {i: int(x[i].varValue) for i in x}
report = pd.DataFrame.from_dict(result, orient='index').reset_index()
report.columns = ['var', 'value']

## Report
The result will show the number of racks, pallets needed for item A, B

In [9]:
report

,var,value
0,r_a,135
1,r_b,818
2,p_a,0
3,p_b,785
